In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gdown
import os
import yaml
import sys
from os.path import abspath, dirname, join
current_dir = os.getcwd()
print(current_dir)
sys.path.insert(0, "/home/yif/Documents/KTH/git/diffusion_policy_escape_energy")

from core.controllers.quadrotor_diffusion_policy import build_networks_from_config
from core.dataset.ee2d_dataset import EscapeEnergy2DDataset
from core.trainers.quadrotor_diffusion_policy_trainer import PlanarQuadrotorDiffusionPolicyTrainer

/home/yif/Documents/KTH/git/diffusion_policy_escape_energy/train


/home/yif/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yif/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


## Load config

In [3]:
with open("../config/config_ee2d.yaml", "r") as file:
    config = yaml.safe_load(file)

## Download dataset

In [4]:
dataset_path = "../dataset_escape_from_u_2d_30000_envs.joblib"
# if not os.path.isfile(dataset_path):
#     gdown.download(id="14safYt34YKn21vL9c_BsRHaG_URPB4Dp", output=dataset_path, quiet=False)

## Load dataset and instantiate the NN

In [5]:
dataset = EscapeEnergy2DDataset(dataset_path=dataset_path, config=config)
print("Dataset loaded successfully.")

net = build_networks_from_config(config)
print("Networks initialized successfully.")

!!!!!! dataset_root dict_keys(['costs', 'paths', 'object_starts', 'ellipse_centers', 'ellipse_radii'])
!!!!!! paths (29868, 20, 2)
!!!!!! object_starts (29868, 2)
!!!!!! ellipse_radii (29868, 3, 2)
!!!!!! ellipse_centers (29868, 3, 2)
----------------------
!!!!!! action (5120, 2)
!!!!!! obs (5120, 2)
!!!!!! obstacle (5120, 12)
----------------------
Dataset loaded successfully.
Networks initialized successfully.


In [ ]:
s = dataset.__getitem__(0)


## Instantiate the trainer

In [7]:
trainer = PlanarQuadrotorDiffusionPolicyTrainer(
    net=net,
    dataset=dataset,
    config=config,
)

!!!!!! len(self.indices) 5376
!!!!!! len(self.indices) 5376


## Start training

In [10]:
trainer.train(num_epochs=5, save_ckpt_epoch=1)
trainer.save_checkpoint("ckpt_final.ckpt")

!!!!!! len(self.indices) 5376


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

!!!!!! len(self.indices) 5376


!!!!!! len(self.indices) 5376
!!!!!! len(self.indices) 5376


Epoch:  20%|██        | 1/5 [00:46<03:05, 46.34s/it, loss=0.983]

...

Epoch:  80%|████████  | 4/5 [03:11<00:48, 48.27s/it, loss=0.324]

!!!!!! len(self.indices) 5376
!!!!!! len(self.indices) 5376


!!!!!! len(self.indices) 5376
!!!!!! len(self.indices) 5376


Epoch: 100%|██████████| 5/5 [04:00<00:00, 48.06s/it, loss=0.207]


!!!!!! len(self.indices) 5376


## Trainnig data visualization
The training data was generated using a MPPI-CBF controller from [shaoanlu/mppi_cbf](https://github.com/shaoanlu/mppi_cbf) repository. 

In [9]:
import joblib
import numpy as np
from matplotlib import pyplot as plt

In [24]:
def visualize_dataset(data):
    """
    Shows 20 samples in the dataset.
    Each subplot illustrates a training trajectory consisting of:
    - The starting position as a green dot;
    - The goal position as a red dot;
    - The positon of the quadrotor is the blue line; and
    - The obstacles are presented as grey circles.
    """
    plt.figure(figsize=(10, 8))
    for i in range(20):
        rand_idx = np.random.randint(len(data["state"]))
        s = data["state"][rand_idx]
        # des_s = data["desired_state"][rand_idx]
        info = data["info"][rand_idx]
        x, y = s[:, 0], s[:, 2]
        # des_x, des_y = des_s[:, 0], des_s[:, 2]
    
        plt.subplot(4, 5, i+1)
        for obs_idx in range(len(info["obs_center"])):
            circle = plt.Circle((info["obs_center"][obs_idx, 0], info["obs_center"][obs_idx, 1]), info["obs_radius"][obs_idx], color='grey', fill=True, linestyle='--', linewidth=2, alpha=0.5)
            plt.gca().add_artist(circle)
        plt.scatter(data["state"][rand_idx][0, 0], data["state"][rand_idx][0, 2], color="green")
        plt.scatter(5, 5, color="red")
        plt.plot(x, y)
        # plt.plot(des_x, des_y, ":")
        plt.grid()
        plt.xlim(-1, 7)
        plt.ylim(-1, 7)
    plt.tight_layout()
    plt.show()

In [25]:
trn_data = joblib.load("../escape_from_u_data.joblib")
visualize_dataset(trn_data)

KeyError: 'state'

<Figure size 1000x800 with 0 Axes>

In [26]:
for key, value in trn_data.items():
    print(f"{key}: {len(value)}")
print('costs', type(trn_data["costs"]))
print('costs', trn_data["costs"])
print('paths', trn_data["paths"])
print('object_starts', trn_data["object_starts"])


costs: 6
paths: 6
object_starts: 6
ellipse_centers: 6
ellipse_radii: 6
costs <class 'numpy.ndarray'>
costs [0.         0.00032877 0.         0.         0.         0.03209365]
paths [[[0.45580925 0.68537036]
  [0.41834608 0.62996103]
  [0.40587738 0.67183557]
  [0.45284282 0.61964025]
  [0.486888   0.60014907]
  [0.54439431 0.65190197]
  [0.60222496 0.67146199]
  [0.63987722 0.61895391]
  [0.65420421 0.6106548 ]
  [0.67334748 0.55435731]
  [0.70662757 0.58796657]
  [0.7679419  0.60811095]
  [0.74965366 0.65252623]
  [0.77285868 0.61885425]
  [0.7803659  0.598214  ]
  [0.7633884  0.55428415]
  [0.77043103 0.52532142]
  [0.79925971 0.49306505]
  [0.79429885 0.43389678]
  [0.75424768 0.43540488]]

 [[0.61691686 0.59070396]
  [0.62100338 0.53397404]
  [0.58773767 0.50142582]
  [0.6541713  0.52666671]
  [0.71135764 0.54752603]
  [0.69974045 0.5578557 ]
  [0.73802118 0.50268235]
  [0.71164017 0.46677389]
  [0.76657945 0.49902675]
  [0.73207873 0.44544288]
  [0.70091109 0.46776482]
  [0.761698